#### Loading libraries

In [1]:
# Load tensorflow
import tensorflow as tf
# Below command is to avoid the known bug which prevents computation on some GPU devices
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)
# Load preprocessing tools
from scipy.ndimage.filters import gaussian_filter
from tensorflow.keras.utils import to_categorical
from PIL import Image
# Load model building blocks
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
# Load pre-trained model library
from tensorflow.keras import applications
# Load miscelaneous libraries
import numpy as np
import os
import matplotlib.pylab as plt
import gc
# Settings to set the max output of numpy to infty
# import sys
# np.set_printoptions(threshold=sys.maxsize)

#### Define image processor and data generator

In [2]:
# Define gaussian blur class
class GaussBlur:
    def __init__(self, radius):
        self.radius = radius
    def blur(self, image):
        return gaussian_filter(image, sigma = self.radius)

In [3]:
# Define class for data generation
class DataGenerator:
    'Generates data for Keras'
    def __init__(self, list_image_paths = None,  
                 dim = None,
                 n_channels = 3, 
                 rescale = 1, 
                 preprocessing_func = None):
        'Initialization'
        self.list_image_paths = list_image_paths
        self.dim = dim
        self.n_channels = n_channels
        self.rescale = rescale
        self.preprocessing_func = preprocessing_func

    def generate(self):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        
        # Initialisation
        X = np.empty((len(self.list_image_paths), *self.dim, self.n_channels))
        
        # Generate data
        for i, image_path in enumerate(self.list_image_paths):
            # Load image and transform
            image = Image.open(os.path.join(image_path))
            if self.dim is not None:
                image = image.resize(self.dim, resample = Image.NEAREST)
            image = np.array(image)[:, :, :self.n_channels]
            image = image * self.rescale
            if self.preprocessing_func is not None:
                image = self.preprocessing_func(image)
            # Store sample
            X[i,] = image

        return X

In [4]:
# Define algorithm for reordering the data based on the index array
def reorder(ind, arr):
    new_arr = np.zeros(arr.shape)
    for i in range(len(arr)):
        new_arr[i] = arr[ind[i]]
    return new_arr  

def order_labels(y_lai, y_lm, y_lat):
    y_lm_reord = dict()
    y_lai_reord = dict()
    y_lat_reord = dict()
    # Perform reording
    for k in range(5):
        i = k + 2
        y_lai_reord[i] = np.sort(y_lai[i], axis = -1) 
        ind_matrix = np.argsort(y_lai[i], axis = -1)
        y_lm_reord[i] = np.array([reorder(ind_matrix[j,:], y_lm[i][j,:]) for j in range(y_lm[i].shape[0])])
        y_lat_reord[i] = np.array([reorder(ind_matrix[j,:], y_lat[i][j,:]) for j in range(y_lat[i].shape[0])])
    return y_lai_reord, y_lm_reord, y_lat_reord

In [5]:
# Define sorter of image names in order by image number (default is alphanumric)
def sorter(item):
    # Since highest marks first, least error = most marks
    radius = float(item[1 : item.find('_')])
    num_img = int(item[item.find('g') + 1 : item.find('j') - 1])
    return (radius, num_img)

#### Load the data

In [6]:
# Extract image paths
img_paths = {}
for k in range(5):
    i = k + 2
    path_prefix = os.path.join(os.getcwd(), 'image_data', 'test', str(i))
    img_paths[i] = [os.path.join(path_prefix, name) for name in sorted(os.listdir(path_prefix), key = sorter)]

# Apply data generators
gaussblur = GaussBlur(1)
params = {'dim': (128, 128), 
          'n_channels': 3, 
          'rescale': 1 / 255, 
          'preprocessing_func': gaussblur.blur
          }
test_generator = {}
for k in range(5):
    i = k + 2
    test_generator[i] = DataGenerator(img_paths[i], **params)
    
# Load images (for all number of forces) as matrix
X = dict()
for k in range(5):
    i = k + 2
    X[i] = test_generator[i].generate()

In [7]:
# Load angle labels
path_prefix = os.path.join(os.getcwd(), 'labels', 'test')
y_lai = dict()
for k in range(5):
    i = k + 2
    path = os.path.join(path_prefix, str(i))
    y_lai[i] = np.load(os.path.join(path,'mags.npy'))

In [8]:
# Load models
models_path = os.path.join(os.getcwd(), 'models')

model_class = load_model(os.path.join(models_path, 'InceptionResNetV2_num_forces.h5'))                       

In [9]:
# predict class
predicted_class = dict()
indices_correct_class = dict()
indices_incorrect_class = dict()
predicted_incorrect_class = dict()
for k in range(5):
    i = k + 2
    predicted_class[i] = model_class.predict(X[i]).argmax(-1)+2
    indices_correct_class[i] = np.argwhere(predicted_class[i] == i).flatten().tolist()
    indices_incorrect_class[i] = np.argwhere(predicted_class[i] != i).flatten().tolist()
    predicted_incorrect_class[i] = predicted_class[i][indices_incorrect_class[i]]

In [10]:
models_ai = dict()                         
for k in range(5):
     i = k + 2
     models_ai[i] = load_model(os.path.join(models_path, 'InceptionResNetV2_mags_'+str(i)+'.h5'))

In [11]:
for k in range(5):
    i = k + 2
    print("Num forces: ", i)
    loss = models_ai[i].evaluate(X[i][indices_correct_class[i], ], y_lai[i][indices_correct_class[i], ])

Num forces:  2
125/125 [==============================] - 5s 40ms/step - loss: 0.0037 - mean_absolute_error: 0.0037
Num forces:  3
125/125 [==============================] - 5s 40ms/step - loss: 0.0208 - mean_absolute_error: 0.0208
Num forces:  4
125/125 [==============================] - 124s 994ms/step - loss: 0.0378 - mean_absolute_error: 0.0378
Num forces:  5
125/125 [==============================] - 134s 1s/step - loss: 0.0352 - mean_absolute_error: 0.0352
Num forces:  6
124/124 [==============================] - 139s 1s/step - loss: 0.0417 - mean_absolute_error: 0.0417


In [12]:
diffs = []
for k in range(5):
    i = k + 2
    print("Num forces: ", i)
    if (len(indices_incorrect_class[i]) > 0):
        for j in range(len(indices_incorrect_class[i])):
            predicted_mags = models_ai[predicted_incorrect_class[i][j]].predict(X[i][None, indices_incorrect_class[i][j], ])
            true_mags = y_lai[i][indices_incorrect_class[i][j], ]
            diff = abs(np.sum(predicted_mags) - np.sum(true_mags))
            diffs.append(diff)
            print("Predicted: ", predicted_mags)
            print("True: ", true_mags)
            print("Difference: ", diff)      

Num forces:  2
Num forces:  3
Num forces:  4
Predicted:  [[0.76645267 0.88127315 1.0849022 ]]
True:  [0.72908997 0.98566124 0.66852237 1.40436357]
Difference:  1.0550092805167846
Num forces:  5
Predicted:  [[0.01686719 0.02409827 0.0208751  0.02136923 0.01936121 0.01986308]]
True:  [0.02380837 0.01757172 0.01935316 0.01808236 0.02317286]
Difference:  0.020445616477124298
Predicted:  [[0.25989276 0.1546325  0.17716785 0.17853102]]
True:  [0.25515731 0.1081789  0.16584061 0.15330085 0.09681757]
Difference:  0.009071141272995797
Predicted:  [[0.26113212 0.15436623 0.18400538 0.18955013]]
True:  [0.25515731 0.1081789  0.16584061 0.15330085 0.09681757]
Difference:  0.009758681267287406
Predicted:  [[0.24547476 0.14580731 0.17487916 0.17727056]]
True:  [0.25515731 0.1081789  0.16584061 0.15330085 0.09681757]
Difference:  0.03586342909953388
Predicted:  [[0.09789623 0.05366202 0.06335273 0.05334696]]
True:  [0.11617361 0.0554022  0.06942852 0.05282908 0.04048098]
Difference:  0.06605644600360

In [13]:
print("Average difference: ", sum(diffs) / len(diffs))

Average difference:  0.19674798197232662


In [14]:
indices_incorrect_class

{2: [],
 3: [],
 4: [1141],
 5: [323, 497, 498, 499, 972, 973, 1115, 1116, 1119, 1459, 1712, 1714, 1715],
 6: [219,
  464,
  465,
  466,
  467,
  2107,
  2108,
  2109,
  2110,
  2111,
  2112,
  2113,
  2114,
  2115,
  2116,
  2117,
  2118,
  3421,
  3422,
  3423,
  3424,
  3426,
  3427,
  3428,
  3429,
  3430,
  3431,
  3432,
  3433,
  3434,
  3435,
  3436,
  3437,
  3438,
  3439]}

In [15]:
predicted_incorrect_class

{2: array([], dtype=int64),
 3: array([], dtype=int64),
 4: array([3]),
 5: array([6, 4, 4, 4, 4, 4, 4, 4, 4, 6, 4, 4, 4]),
 6: array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])}